## Task

### 1. Create a new notebook for this task. Be sure to import the relevant libraries, along with your ords_prods_merge dataframe, which should include your newly derived columns from the previous Exercise.

In [3]:
import pandas as pd
import numpy as np
import os

In [4]:
path = r'C:\Users\Víctor A\Desktop\CAREER FOUNDRY\Instacart Basket Analysis'

In [5]:
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge_4.8_cleaned.pkl'))

In [9]:
ords_prods_merge.shape

(32399732, 20)

### 2. In this Exercise, you learned how to find the aggregated mean of the “order_number” column grouped by “department_id” for a subset of your dataframe. Now, repeat this process for the entire dataframe.

In [11]:
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


### 3. Analyze the result. How do the results for the entire dataframe differ from those of the subset? Include your comments in a markdown cell below the executed code

#### Since we have now calculated the average of the entire dataframe, the numbers differ. They are not very big differences.

### 4. Follow the instructions in the Exercise for creating a loyalty flag for existing customers using the transform() and loc() functions.

#### Already executed and saved in the dataframe during the exercise instructions.

In [13]:
ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,is_first_order,add_to_cart_order,reordered,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,False,5,0,Mid-range product,Regularly busy,Regularly busy,Most orders,32,Regular customer
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,False,1,1,Mid-range product,Regularly busy,Regularly busy,Average orders,32,Regular customer
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,False,20,0,Mid-range product,Busiest day,Busiest day,Average orders,5,New customer
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,0.0,True,10,0,Mid-range product,Regularly busy,Least busy,Most orders,3,New customer
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,False,11,1,Mid-range product,Least busy,Least busy,Average orders,3,New customer


### 5. The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [15]:
ords_prods_merge.groupby('loyalty_flag').agg({'prices' : ['count', 'mean', 'std', 'min', 'max']})

prices                               
                     count      mean       std  min   max
loyalty_flag                                             
Loyal customer    10282763  7.773575  4.206151  1.0  25.0
New customer       6242841  7.801206  4.266834  1.0  25.0
Regular customer  15874128  7.798262  4.254886  1.0  25.0

### 6. The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria:

    If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
    If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”


In [21]:
# Create column 'average spend'
ords_prods_merge['average_spend'] = ords_prods_merge.groupby('user_id')['prices'].transform('mean')

In [29]:
# Create flag column based on criteria
ords_prods_merge['spending_flag'] = 'Low spender'
ords_prods_merge.loc[ords_prods_merge['average_spend'] >= 10, 'spending_flag'] = 'High spender'

In [31]:
# Check spender flags
ords_prods_merge['spending_flag'].value_counts(dropna = False)

spending_flag
Low spender     32280013
High spender      119719
Name: count, dtype: int64

### 7. In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:

    If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
    If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
    If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”


In [34]:
# Create column  'customer frequency' according to the median in the “days_since_prior_order” column.

ords_prods_merge['customer_frequency'] = ords_prods_merge.groupby('user_id')['days_since_last_order'].transform('median')

In [40]:
# Create flag column based on criteria

ords_prods_merge['frequency_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['customer_frequency'] > 20, 'frequency_flag'] = 'Non-frequent customer'
ords_prods_merge.loc[ords_prods_merge['customer_frequency'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [42]:
# Check frequency flags

ords_prods_merge['frequency_flag'].value_counts(dropna = False)

frequency_flag
Frequent customer        22793177
Regular customer          6920582
Non-frequent customer     2685973
Name: count, dtype: int64

### 9. Export your dataframe as a pickle file and store it correctly in your “Prepared Data” folder.

In [51]:
# Checking new dataframe
ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_spend,spending_flag,customer_frequency,frequency_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,Mid-range product,Regularly busy,Regularly busy,Most orders,32,Regular customer,6.935811,Low spender,8.0,Frequent customer
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,Mid-range product,Regularly busy,Regularly busy,Average orders,32,Regular customer,6.935811,Low spender,8.0,Frequent customer
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,Mid-range product,Busiest day,Busiest day,Average orders,5,New customer,7.930208,Low spender,6.0,Frequent customer
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,Mid-range product,Regularly busy,Least busy,Most orders,3,New customer,4.972414,Low spender,9.0,Frequent customer
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,Mid-range product,Least busy,Least busy,Average orders,3,New customer,4.972414,Low spender,9.0,Frequent customer


In [49]:
ords_prods_merge.shape

(32399732, 24)

In [53]:
# Export dataframe as a pickle file
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_merge_4.8.2_cleaned.pkl'))